In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
from hanlp.components.pos import RNNPartOfSpeechTagger

tagger1 = RNNPartOfSpeechTagger()
save_moedl='../model/pos1030first/'
tagger1.load(save_moedl)
tagger2 = RNNPartOfSpeechTagger()
save_moedl='../model/pos1030second/'
tagger2.load(save_moedl)
tagger3 = RNNPartOfSpeechTagger()
save_moedl='../model/pos110202/'
tagger3.load(save_moedl)

In [7]:
def Process(addr, tag):
    strList = []
    tagCur = ''
    for i in range(len(tag)):
        if tag[i] != 'X' and tag[i] != 'M':
            if i != 0:
                strList.append(',')
                strList.append(tagCur)
                strList.append(',')
            strList.append(addr[i])
            tagCur = tag[i]
        else:
            strList.append(addr[i])
    strList.append(',')
    strList.append(tagCur)
    return ''.join(strList)

In [8]:
def predict(addStr):
    addList = [list(addStr)]
    init = tagger1.predict(addList)[0]
    strNextList = []
    start = -1
    for i in range(len(addList[0])):
        if init[i] == 'RX':
            start = i
            break
    if start == -1:
        return ' '.join(init[:])
    else:
        second = tagger2.predict([addList[0][start:]])[0]
        return ' '.join(init[:start]) + ' ' + ' '.join(second)

In [9]:
def batchReadFiles(rdAddress):
    with open(rdAddress, 'r') as f:
        for item in f:
            address=list(item[:-1])
            M1 = tagger1.predict(address)
            address1 = Process(address,M1)
            M2 = predict(address)
            M2 = M2.split()
            address2 = Process(address,M2)
            print('原始地址：', item.strip())
            print('一级模型：', address1)
            print('二级模型：', address2)
            M3 = tagger3.predict(address)
            address3 = Process(address,M3)
            print('未分级模型：', address3+'\n')

In [10]:
batchReadFiles('test4.txt')

原始地址： 安徽省六安市霍邱县城关地税分局安徽六安霍邱县城关镇光明大道
一级模型： 安徽省,R1,六安市,R2,霍邱县,R3,城关地税分局,R4,安徽,R1,六安,R2,霍邱县,R3,城关镇,R4,光明大道,RX
二级模型： 安徽省,R1,六安市,R2,霍邱县,R3,城关地税分局,R4,安徽,R1,六安,R2,霍邱县,R3,城关镇,R4,光明大道,R5
未分级模型： 安徽省,R1,六安市,R2,霍邱县,R3,城关地税分局,R4,安徽,R1,六安,R2,霍邱县,R3,城关镇,R4,光明大道,R5

原始地址： 北京市北京市市辖区东城区深圳市龙岗坂田华为基地A10
一级模型： 北京市,R1,北京市,R1,市辖区,R4,东城区,R4,深圳市,R2,龙岗,R4,坂田,RX,华为基地A10,RX
二级模型： 北京市,R1,北京市,R1,市辖区,R4,东城区,R4,深圳市,R2,龙岗,R4,坂田华为基地,R31,A10,R7
未分级模型： 北京市,R1,北京市市辖区,R1,东城区,R3,深圳市,R2,龙岗,R3,坂田华为基地,R31,A10,R7

原始地址： 广东省佛山市南海区广东省佛山市南海区狮山镇罗村乐安开发区中一
一级模型： 广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,狮山镇,R4,罗村乐安开发区中一,RX
二级模型： 广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,狮山镇,R4,罗村乐安开发区中一,R5
未分级模型： 广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,狮山镇,R4,罗村乐安开发区中一,R5

原始地址： 福州达北农生物技术有限公司贵州省贵阳市黔灵山路T6号金谷苑A3
一级模型： 福州,R2,达北农生物技术有限公司,R4,贵州省,R1,贵阳市,R2,黔灵山路T6号金谷苑A3,RX
二级模型： 福州,R2,达北农生物技术有限公司,R4,贵州省,R1,贵阳市,R2,黔灵山路,R5,T6号,R6,金谷苑,R30,A3,R7
未分级模型： 福州达北农生物技术有限公司,R3,贵州省,R1,贵阳市,R2,黔灵山路,R5,T6号,R6,金谷苑,R30,A3,R7

原始地址： 甘肃省酒泉市玉门市新市区老城水管处甘肃省疏勒河流勘水资源管理
一级模型： 甘肃省,R

原始地址： 杭州意法服饰城张金苗服饰店
一级模型： 杭州意法服饰城张金苗服饰店,RX
二级模型： 杭州意法服饰城,R25,张金苗服饰店,R25
未分级模型： 杭州意法服饰城张金苗服饰店,R25

原始地址： 浙江省台州市天台县浙江省天台县白鹤镇西工业园区东园路5号永贵
一级模型： 浙江省,R1,台州市,R2,天台县,R3,浙江省,R1,天台县,R3,白鹤镇,R4,西工业园区东园路5号永贵,RX
二级模型： 浙江省,R1,台州市,R2,天台县,R3,浙江省,R1,天台县,R3,白鹤镇,R4,西工业园区,R31,东园路,R5,5号,R6,永贵,R21
未分级模型： 浙江省,R1,台州市,R2,天台县,R3,浙江省,R1,天台县,R3,白鹤镇,R4,西工业园区东园路,R31,5号,R6,永贵,R31

原始地址： 甘肃省嘉峪关市长城区建设社区服务中心长城区建设社区
一级模型： 甘肃省,R1,嘉峪关市,R2,长城区,R3,建设社区服务中心长城区建设社区,RX
二级模型： 甘肃省,R1,嘉峪关市,R2,长城区,R3,建设社区,R5,服务中心,R31,长城区建设社区,R5
未分级模型： 甘肃省,R1,嘉峪关市,R2,长城区,R24,建设社区服务中心长城区建设社区,R5

原始地址： 界首市双马机械有限公司安徽省界首市鸭王工业园区中心路室
一级模型： 界首市,R3,双马机械有限公司,R4,安徽省,R1,界首市,R3,鸭王工业园区中心路室,RX
二级模型： 界首市,R3,双马机械有限公司,R4,安徽省,R1,界首市,R3,鸭王工业园区,R31,中心路,R5,室,R90
未分级模型： 界首市双马机械有限公司安徽省界首市鸭王工业园区中心路室,R20

原始地址： 江西省吉安市峡江县东鹏瓷砖店江西省吉安市峡江县水边镇玉峡大
一级模型： 江西省,R1,吉安市,R2,峡江县,R3,东鹏瓷,R4,砖店,R4,江西省,R1,吉安市,R2,峡江县,R3,水边镇,R4,玉峡大,RX
二级模型： 江西省,R1,吉安市,R2,峡江县,R3,东鹏瓷,R4,砖店,R4,江西省,R1,吉安市,R2,峡江县,R3,水边镇,R4,玉峡大,R5
未分级模型： 江西省,R1,吉安市,R2,峡江县,R3,东鹏瓷砖店,R4,江西省,R1,吉安市,R2,峡江县,R3,水边镇,R4,玉峡大,R99

原始地址： 江苏省淮安市

原始地址： 中国黑龙江齐齐哈尔富拉尔基黑龙江省齐齐哈尔市富拉尔基重机厂
一级模型： 中国,R3,黑龙江,R1,齐齐哈尔,R2,富拉尔基,R3,黑龙江省,R1,齐齐哈尔市,R2,富拉尔基重机厂,RX
二级模型： 中国,R3,黑龙江,R1,齐齐哈尔,R2,富拉尔基,R3,黑龙江省,R1,齐齐哈尔市,R2,富拉尔基重机厂,R20
未分级模型： 中国,R4,黑龙江,R1,齐齐哈尔,R2,富拉尔基,R3,黑龙江省,R1,齐齐哈尔市,R20,富拉尔基重机厂,R20

原始地址： 广东省佛山市佛山市顺德区大良趣步户外有限
一级模型： 广东省,R1,佛山市,R2,佛山市,R2,顺德区,RX,大良趣步户外有限,RX
二级模型： 广东省,R1,佛山市,R2,佛山市,R2,顺德区大良趣步户外有限,R20
未分级模型： 广东省,R1,佛山市,R2,佛山市,R3,顺德区,R20,大良趣步户外有限,R20

原始地址： 河南郑州中原区河南省扶沟县园区2号路河南省扶沟县职业教育中心
一级模型： 河南,R1,郑州,R2,中原区,R3,河南省,R1,扶沟县,R3,园区2号路河南省扶沟县职业教育中心,RX
二级模型： 河南,R1,郑州,R2,中原区,R3,河南省,R1,扶沟县,R3,园区,R5,2号路,R6,河南省扶沟县职业教育中心,R20
未分级模型： 河南,R1,郑州,R2,中原区,R3,河南省,R1,扶沟县园区,R3,2号路,R5,河南省扶沟县职业教育中心,R20

原始地址： 河南省洛阳市伊川县洛阳市伊川县华美建材城b区13号楼二楼
一级模型： 河南省,R1,洛阳市,R2,伊川县,R3,洛阳市,R2,伊川县,R3,华美建材城b区13号楼二楼,RX
二级模型： 河南省,R1,洛阳市,R2,伊川县,R3,洛阳市,R2,伊川县,R3,华美建材城,R25,b区,R6,13号楼,R6,二楼,R7
未分级模型： 河南省,R1,洛阳市,R2,伊川县,R3,洛阳市,R2,伊川县华美建材城,R3,b区,R6,13号楼,R6,二楼,R7

原始地址： 浙江杭州浙江省杭州市下沙经济开发区上沙路与华景街交叉口龙湖
一级模型： 浙江,R1,杭州,R2,浙江省,R1,杭州市,R2,下沙经济开发区,R4,上沙路与华景街交叉口龙湖,RX
二级模型： 浙江,R1,杭州,R2,浙江省,R1,杭州市,R2,下沙经济开发区,R4,上

原始地址： 陕西省榆林市子州县中医院家属,院,
一级模型： 陕西省,R1,榆林市,R2,子州县,RX,中医院家属,院,,RX
二级模型： 陕西省,R1,榆林市,R2,子州县中医院家属,院,,R22
未分级模型： 陕西省,R1,榆林市,R2,子州县,R3,中医院家属,院,,R30

原始地址： 英额门镇英额门村河北13-2号
一级模型： 英额门镇,R4,英额门村河北13-2号,RX
二级模型： 英额门镇,R4,英额门村河北13-2号,R5
未分级模型： 英额门镇,R4,英额门村河北,R5,13-2号,R6

原始地址： 重庆市重庆市长寿区重庆市长寿区洪湖镇卫生院
一级模型： 重庆市,R1,重庆市,R1,长寿区,R3,重庆市,RX,长寿区,R3,洪湖镇卫生院,R4
二级模型： 重庆市,R1,重庆市,R1,长寿区,R3,重庆市长寿区洪湖镇卫生院,R22
未分级模型： 重庆市,R1,重庆市,R1,长寿区,R3,重庆市,R1,长寿区,R22,洪湖镇卫生院,R4

原始地址： 河南省信阳市河区河南省信阳市农行中山北路分理处205号
一级模型： 河南省,R1,信阳市,R2,河区,R3,河南省,R1,信阳市农行中山北路分理处205号,RX
二级模型： 河南省,R1,信阳市,R2,河区,R3,河南省,R1,信阳市农行中山北路分理处,R23,205号,R6
未分级模型： 河南省,R1,信阳市,R2,河区,R3,河南省,R1,信阳市农行中山北路分理处,R23,205号,R6

原始地址： 山东滨州市惠民县惠民县大济路车管所对面吉利汽车
一级模型： 山东,R1,滨州市,R2,惠民县,R3,惠民县,R3,大济路车管所对面吉利汽车,RX
二级模型： 山东,R1,滨州市,R2,惠民县,R3,惠民县,R3,大济路,R5,车管所,R24,对面吉利汽车,R90
未分级模型： 山东,R1,滨州市,R2,惠民县,R3,惠民县,R24,大济路车管所,R5,对面,R90,吉利汽车,R25

原始地址： 贵州省贵阳市贵安新区栋青南路贵州师范大学花溪校区
一级模型： 贵州省,R1,贵阳市,R2,贵安新区,R3,栋青南路贵州师范大学花溪校区,RX
二级模型： 贵州省,R1,贵阳市,R2,贵安新区,R3,栋青南路,R5,贵州师范大学花溪校区,R21
未分级模型： 贵州省,R1,贵阳市,R2,贵安新区,R3,栋青南路,